<a href="https://youtu.be/A2euuevpYis" target="_parent"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/09/YouTube_full-color_icon_%282017%29.svg/71px-YouTube_full-color_icon_%282017%29.svg.png" alt="Open In Colab"/></a>


# **Práctico Sistemas Recomendadores: pyreclab - Slope One**

En este práctico seguiremos utilizando [pyreclab](https://github.com/gasevi/pyreclab), con el cual estamos aprendiendo distintas técnicas de recomendación. Seguiremos usando la misma base de datos de los prácticos anteriores, para que puedan comparar los métodos y sus implementaciones. Este práctico está acompañado de un [video comentando la actividad](https://youtu.be/A2euuevpYis).

En esta oportunidad exploraremos el recomendador de Pendiente Uno o **Slope One** [1].

**Adaptado y preparado por:** Francisca Cattan 📩 fpcattan@uc.cl

Referencias 📖
------
[1] *Lemire, D., & Maclachlan, A. (2005, April). Slope One Predictors for Online Rating-Based Collaborative Filtering. In SDM (Vol. 5, pp. 1-5).*


**Nombre**:  completa tu nombre aquí :D

## Actividad 1 👓

Antes de empezar con el práctico, responde la siguiente pregunta con lo visto en clases.

**Pregunta:** Explique cómo funciona Slope One (como modelo teórico, no piense en la implementación). En particular explique:

- Repasemos: ¿Por qué este recomendador es un algoritmo de Filtrado Colaborativo?
- Este Filtrado Colaborativo, ¿está basado en el usuario o en los items? ¿Por qué?
- ¿Qué datos recibe Slope One y qué hace con ellos? (qué tipo de columnas y qué calculo)
- ¿Qué pasaría si se agrega un nuevo rating a la base de datos?
- Opcional: ¿Cómo crees que le iría al recomendador con un usuario que acaba de entrar al sistema y ha asignado muy pocos ratings?

💡 *Hint: La bibliografía todo lo puede.*

**Respuesta:**

- Porque utiliza los ratings que han evaluado otros usuarios dentro del sistema para hacer una prediccion del rating que le daria el usuario a partir de la comparacion entre sus calificaciones ya hechas en ciertos items con los demas usuarios. De este forma, se utiliza la colaboracion de multiples usuarios para hacer una prediccion de rating que dara un usuario sobre un item cualquiera.

- Este algoritmo esta basado en los items, ya que no filtra a los usuarios sino que utiliza a todos. De esta forma solo le importa la informacion de los ratings de ciertos items y no discrepa sobre que usuario ingreso cada rating (Solo hace una interseccion entre usuarios que utilizaron ciertos items).

-  _Slope One_ recibe columnas que representan todos las calificaciones de cierto usuario donde cada fila representa un item, luego para calcular una prediccion de cierto item *i* para un usuario *u* debe utilizar los raitings previos de *u* y compararlos con otros usuarios que hayan consumido ese item i e hayan consumido otros items en comun con *u*. Para esto se calcula la desviacion promedio entre los dos items, lo que se puede hacer multiples veces con mas items en comun para disminuir el error.

- En el caso de que se agregue un nuevo rating a la base de datos, se debe calcular denuevo la desviacion promedio de ese item en especifico y se promedia nuevamente con los demas datos, por lo que no es tan complejo y lento de realizar.

- Al tener el usuario un numero bajo de raitin, es muy probable que las recomendaciones no sean tan efectivas debido que son basadas en muy pocos datos. Esto porque solo podriamos comparar una pequena muestra de items que el usuario a consumido, lo que es bastante propenso a obtener errores.




# **Configuración Inicial**

## Paso 1:
Descargue directamente a Colab los archivos del dataset ejecutando las siguientes 3 celdas:


In [6]:
!curl -L -o "u1.base" "https://drive.google.com/uc?export=download&id=1bGweNw7NbOHoJz11v6ld7ymLR8MLvBsA"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    380      0 --:--:--  0:00:01 --:--:--   380
100 1546k  100 1546k    0     0  1327k      0  0:00:01  0:00:01 --:--:--  167M


In [7]:
!curl -L -o "u1.test" "https://drive.google.com/uc?export=download&id=1f_HwJWC_1HFzgAjKAWKwkuxgjkhkXrVg"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    562      0 --:--:-- --:--:-- --:--:--   562
100  385k  100  385k    0     0   465k      0 --:--:-- --:--:-- --:--:--  465k


In [8]:
!curl -L -o "u.item" "https://drive.google.com/uc?export=download&id=10YLhxkO2-M_flQtyo9OYV4nT9IvSESuz"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    780      0 --:--:-- --:--:-- --:--:--   780
100  230k  100  230k    0     0   357k      0 --:--:-- --:--:-- --:--:--  357k


Los archivos **u1.base** y **u1.test** tienen tuplas {usuario, item, rating, timestamp}, que es la información de preferencias de usuarios sobre películas en una muestra del dataset [movielens](https://grouplens.org/datasets/movielens/).

## Paso 2:

Instalamos pyreclab utilizando pip.

In [9]:
!pip install pyreclab --upgrade

## Paso 3:

Hacemos los imports necesarios para este práctico.

In [10]:
import pyreclab
import numpy as np
import pandas as pd

# **El dataset**

💡 *En prácticos anteriores, vimos como analizar este dataset. Puedes revisarlos en caso de dudas.*

## Paso 4:

Ya que queremos crear una lista de recomendación de items para un usuario en especifico, necesitamos obtener información adicional de cada película tal como título, fecha de lanzamiento, género, etc. Cargaremos el archivo de items descargado "u.item" para poder mapear cada identificador de ítem al conjunto de datos que lo describe.

In [11]:
# Definimos el orden de las columnas
info_cols = [ 'movieid', 'title', 'release_date', 'video_release_date', 'IMDb_URL', \
              'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', \
              'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', \
              'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western' ]

# Asignamos a una variable la estructura de datos de los items
info_file = pd.read_csv('u.item', sep='|', index_col = 0, names = info_cols, header=None, encoding='latin-1')

# **Slope One**

## Paso 5:

Seguiremos un camino muy similar a los ejercicios de User KNN e Item KNN. Crearemos una instancia del algoritmo de recomendación y luego pasaremos a la fase de entrenamiento.

In [12]:
# Declaramos la instancia SlopeOne
mySlopeOne = pyreclab.SlopeOne(dataset='u1.base', dlmchar=b'\t', header=False, usercol=0, itemcol=1, ratingcol=2)

In [17]:
# Y enntrenamos
mySlopeOne.train(progress = True)

## Actividad 2 👓

**Pregunta:** Explique qué hace el método `train()` en este caso, dado el modelo teórico. ¿Calcula información?, ¿no hace nada?, ¿ordena los datos? 

**Respuesta:**

Calcula la desviacion promedio entre todos los pares de items que se encuentran en la base de datos, de esta forma no es necesarion calcularlo cuando se quiere hacer la prediccion y solo se debe sumar y dividir entre los items en comun que tiene el usuario y el item al que se quiere hacer la prediccion.

## Paso 6:

Llego la hora de predecir el rating.

In [14]:
# Esta es la predicción de rating que el usuario ID:457 otorgaría al ítem ID:37
# De esta forma podemos comparar el resultado con los prácticos anteriores
mySlopeOne.predict("457", "37")

3.2408759593963623

In [15]:
# También podemos guardar la predicción en una variable
prediction = mySlopeOne.predict("457", "37")

In [16]:
# Podemos comprobar las peliculas rankeadas por el usuario ID:457
# Que ciertamente ha participado activamente (¡156 items!)
train_file = pd.read_csv('u1.base', sep='\t', names = ['userid', 'itemid', 'rating', 'timestamp'], header=None)
train_file[train_file['userid'] == 457]

,userid,itemid,rating,timestamp
37269,457,1,4,882393244
37270,457,7,4,882393278
37271,457,9,5,882393485
37272,457,11,4,882397020
37273,457,13,3,882393883
...,...,...,...,...
37420,457,1047,2,882395964
37421,457,1119,4,882398308
37422,457,1168,5,882548761
37423,457,1210,4,882549905


In [18]:
# Y también cuáles usuarios han rankeado la pelicula ID:37
train_file[train_file['itemid'] == 37]

,userid,itemid,rating,timestamp
1302,13,37,1,882397011
14851,201,37,2,884114635
19670,268,37,3,876514002
29489,363,37,2,891498510
31084,385,37,4,880013483
32996,405,37,1,885548384
62777,773,37,3,888540352


## Actividad 3 👓

Haremos un pequeño experimento para entender mejor como funciona Slope One. Gracias al ejercicio anterior, sabemos que el usuario 457 ya ha asignado el mejor rating (5 ⭐) a las dos peliculas ID:9 e ID:1168. Comparemos.

**Pregunta:** ¿Cómo se explican estos resultados?  

**Respuesta:**

In [19]:
prediction_id9 = mySlopeOne.predict("457", "9")
prediction_id1168 = mySlopeOne.predict("457", "1168")

print('Prediction for ID:9 :', prediction_id9)
print('Prediction for ID:1168 :', prediction_id1168)

Prediction for ID:9 : 4.385906219482422
Prediction for ID:1168 : 4.206681251525879


## Paso 7:

Generaremos ahora una lista ordenada de las top-N recomendaciones, dado un usuario.



In [20]:
# Mediante el método recommend() genereremos una lista top-5 recomendaciones para el usuario ID:457
reclist_slopeone = mySlopeOne.recommend("457", 5)

# Y visualizaremos el resultado
print('Lista de items según ID:', reclist_slopeone)

Lista de items según ID: ['1592', '1589', '1656', '1431', '1653']


In [21]:
# Lo convertimos a numpy array
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
1592                               Magic Hour, The (1998)
1589                                   Schizopolis (1996)
1656                                   Little City (1998)
1431                                  Legal Deceit (1997)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

## Actividad 4 👩🏻‍💻

Genera una nueva recomendacion, modificando los hiperparametros de usuario y topN a tu elección.

**Pregunta:** ¿Ves una diferencia en la recomendación entre el nuevo usuario y el usuario ID:457?

**Respuesta:**

In [23]:
# Mediante el método recommend() genereremos una lista top-10 recomendaciones para el usuario ID:23
reclist_slopeone = mySlopeOne.recommend("23", 10)

# Y visualizaremos el resultado
print('Lista de items según ID:', reclist_slopeone)

Lista de items según ID: ['1653', '1536', '1500', '1463', '1367', '108', '107', '106', '104', '103']


Vemos que las recomendaciones son completamente diferentes, esto se debe a que el algoritmo _slope one_ es un algoritmo personalizado para cada usuario, ya que, compara las peliculas que el usuario objetivo vio como punto de comparacion con el resto de usuarios. Esto cambia para cada uno de los usuarios.

## Actividad 5 👩🏻‍💻

Dado el usuario ID:44, cree dos listas de películas recomendadas; la primera utilizando el algoritmo Most Popular y la segunda utilizando el algoritmo Slope One.

**Pregunta:** Realice un analisis apreciativo de las similitudes y diferencias entre ambas recomendaciones.

**Respuesta:**

_Slope One_

In [30]:
# Mediante el método recommend() de Slope One genereremos una lista top-5 recomendaciones para el usuario ID:44
reclist_slopeone = mySlopeOne.recommend("44", 5)

# Y visualizaremos el resultado
print('Lista de items según ID:', reclist_slopeone)

Lista de items según ID: ['1656', '1064', '1625', '1599', '1512']


_Most Popular_

In [39]:
# Definicion de objeto "most popular"
most_popular = pyreclab.MostPopular(dataset='u1.base',
                   dlmchar=b'\t',
                   header=False,
                   usercol=0,
                   itemcol=1,
                   ratingcol=2)

# en teoria no es "entrenar" , objeto "most_popular" solo registra los items más populares en el set de entrenamiento  
most_popular.train()

In [42]:
# Mediante el método recommend() de Most Popular genereremos una lista top-5 recomendaciones para el usuario ID:44

ranking = [int(r) for r in most_popular.recommend("44", 5, includeRated=False)]
print('Lista de items según ID:', ranking)

Lista de items según ID: [50, 100, 181, 286, 1]


In [43]:
# Mediante el método recommend() de Most Popular genereremos una lista top-5 recomendaciones para el usuario ID:44

ranking = [int(r) for r in most_popular.recommend("23", 5, includeRated=False)]
print('Lista de items según ID:', ranking)

Lista de items según ID: [50, 100, 258, 181, 294]


Vemos que las recomendaciones son completamente diferentes entre los dos metodos implementados, este es porque  _most popular_ solo toma en cuenta los items mas consumidos entre todos los usuarios (Dando la misma recomendacion a todos los usuarios), mientras que _slope one_ hace una prediccion personalizada para cada usuario a partir de las peliculas que ya califico.